In [ ]:
!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-geometric


In [1]:
cd drive/My\ Drive/ElPhotons/

/content/drive/My Drive/ElPhotons


In [2]:
import h5py
f = h5py.File("SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5",'r')

In [ ]:
for key in f.keys():
    print(key) #Names of the groups in HDF5 file.

X
y


In [3]:
#Get the HDF5 group
jet = f['X']

In [5]:
import torch
feats=torch.from_numpy(jet.value).cuda()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  


In [14]:
#Get the HDF5 group
ep = f['y']

In [15]:
labels = torch.from_numpy(ep.value).cuda()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  """Entry point for launching an IPython kernel.


In [16]:
import torch_geometric.transforms
from torch_geometric.nn import knn_graph
import torch_geometric.data
import torch 
from torch_geometric.data import Data
import numpy as np
def jets(datei,number1,number2):

    cols = None

    graphs=[]

    for i in range(number1,number2):

      ecal=datei[i][:,:,0]

      xhit2,yhit2=torch.nonzero(ecal,as_tuple=True)  ## Select hits in detector

      eneEcal=ecal[xhit2,yhit2] ## Select energies of hits

      feats=torch.stack((xhit2.float(),yhit2.float(),eneEcal),dim=1) ## concatenate x,y locations and energies (3 features in total)

      cords=feats[:,[0,1]].cuda() ## cords = x,y coordinates 

      if cords.shape[0]>=10: ## just to make sure a sample is not empty 

        edge_index = knn_graph(cords, k=4, batch=None, loop=True,num_workers=10)  ## Create knn graph adjacency matrix 

        donnees=Data(x=feats,edge_index=edge_index,y=labels[i]) ## Create graph data with feature matrix x and adjacency matrix edge_index
        
        graphs.append(donnees)

    return graphs

In [17]:
test = jets(feats,0,100000)


In [11]:
torch.save(test,'test.pt')